### Load Dependencies

In [1]:
%pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
import opendatasets as od
import torch
import torchvision
from torch.utils.data import Subset
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split

### Load Dataset

In [3]:
# Download Kaggle dataset (Kaggle username and key is required)
# {"username":"christopherconroy","key":"1915e76943ae798bc236fb7c2de6d28d"}
od.download('https://www.kaggle.com/datasets/grassknoted/asl-alphabet')

Skipping, found downloaded files in ".\asl-alphabet" (use force=True to force download)


In [4]:
# Constants
TEST_DATA_DIR = 'asl-alphabet/asl_alphabet_test/asl_alphabet_test'
TRAIN_DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
TRAIN_SPLIT = 0.8  # Fraction of train data in train/valid split

In [5]:
# Load dataset
dataset = torchvision.datasets.ImageFolder(
    root=TRAIN_DATA_DIR,
    transform=ToTensor()
)

# Perform stratified split of dataset indicies
dataset_inds = range(0, len(dataset))
train_inds, valid_inds = train_test_split(dataset_inds, train_size=TRAIN_SPLIT, stratify=dataset.targets)

# Create training and validation subsets
train_set = Subset(dataset, train_inds)
valid_set = Subset(dataset, valid_inds)